# Dog breed identification competition

To do:
* Create train and validation set

* Train VGG16 model and validate
    * Finetune to dog breed dataset
    * Figure out what additional pre-processing steps can be done to improve accuracy
    
* Test on test set 

* submit

In [4]:
from __future__ import division,print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import utils; reload(utils)
from utils import plots
from vgg16 import Vgg16

In [19]:
main_dir = %pwd

### Create validation set

In [4]:
# %mkdir valid

mkdir: cannot create directory ‘valid’: File exists


In [27]:
# os.chdir(main_dir+'/train/')
# all_train = glob('*jpg')
# valid_set_fns = np.random.choice(all_train, int(len(all_train)*.3), replace = False)
# [os.rename(main_dir+'/train/'+x, main_dir+'/valid/'+x) for x in valid_set_fns];

### Initialize Vgg16 model and batches generator

In [ ]:
vgg = Vgg16()

In [20]:
batch_size = 64
num_epochs = 3
batches = vgg.get_batches(main_dir + '/train', batch_size = batch_size)
val_batches = vgg.get_batches(main_dir + '/valid', batch_size = batch_size*2)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


### Train model and save weights

In [17]:
for i in range(3):
    vgg.fit(batches,val_batches,nb_epoch = 1)
    latest_weights_filename = 'ft'+str(i)
    vgg.model.save_weights(main_dir+'/'+latest_weights_filename)

NameError: name 'val_batches' is not defined

In [15]:
??vgg.fit

In [14]:
??vgg